# BLISS walkthrough

The following steps are shown in this notebook:

0. install dependencies
1. load file from disk
2. visualize data before any manipulation (rough sanity check we're interpreting files correctly)
3. set mask bits using various flaggers (spectral kurtosis and filter roll-off)
4. estimate noise statistics (noise floor + noise power)
5. integrate along different drift rates
6. :-|

In [ ]:
# These are always good to have loaded and ipympl makes the interactive plots work

!pip install numpy matplotlib ipympl
# h5py hdf5plugin  scipy


The following will assume this is in something like the "notebooks" folder inside bliss, and everything is already built in the `build/` directory of bliss

In [ ]:
import sys
if "../build/bliss/python" not in sys.path:
    sys.path.append("../build/bliss/python/")

import pybliss
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt

# don't need these where we're going!
# import h5py
# import hdf5plugin

Set these locations and paths according to specific data to look at...

In [ ]:
data_loc = "/home/nathan/datasets/voyager_2020_data/"
test_file = f"{data_loc}/single_coarse_guppi_59046_80036_DIAG_VOYAGER-1_0011.rawspec.0000.h5"


In [ ]:
# We've got an internal structure that reads hdf5 and dumps it to a neat class without exposing hdf5 much
# in c++, that class can take an already open hdf5 and I'd like to support that from h5py as well one day....
# for now, just give a path to an hdf5 file
# The more pressing thing is going to be reading in whole cadences at once
filterbank_file = pybliss.filterbank_data(test_file)


## Visualize data from disk

This shows reading an ndarray we read from disk in our own compute format and sharing it with numpy. The `from_dlpack` method is zero-copy. When the cuda backend comes in to town, this obviously won't work for CUDA-backed ndarrays, but you'll be able to do explicit data movements to copy to cpu, load to numpy, or load it to cupy for cuda operations.

In [ ]:
data_arr = np.from_dlpack(filterbank_file.data)
mask_arr = np.from_dlpack(filterbank_file.mask)

plt.figure()
plt.plot(np.mean(data_arr, 0))

plt.figure()
plt.plot(np.sum(mask_arr, 0))



In [ ]:


# sec_per_spectra = abs(1 / (1e6*filterbank_file.foff))
# N = int(filterbank_file.tsamp / sec_per_spectra)
# M = filterbank_file.data.size(0)
# print(N)

# sk = pybliss.estimators.estimate_spectral_kurtosis(filterbank_file.data, 51, M)

# plt.figure()
# plt.title("Spectral Kurtosis across Channel")
# plt.plot(np.from_dlpack(sk))

# plt.plot(np.from_dlpack(sk) > 2)
# plt.plot(np.from_dlpack(sk) < 0.05)

In [ ]:


flagged_filterbank = pybliss.flaggers.flag_spectral_kurtosis(filterbank_file, 0.05, 5)

flagged_filterbank = pybliss.flaggers.flag_filter_rolloff(filterbank_file, 0.20)



In [ ]:
mask = np.from_dlpack(flagged_filterbank.mask)

print(mask.sum())
print(mask.shape)

plt.figure()
plt.title("Flags across channel")
plt.plot(mask.mean(0))

In [ ]:
noise_est_slice = data_arr[:, 200000:500000]

print(noise_est_slice.shape)


In [ ]:
print(np.mean(noise_est_slice))
m = pybliss.bland.mean(np.array(noise_est_slice))
print(np.from_dlpack(m))

print(np.sum(noise_est_slice) / noise_est_slice.size)

# numpy: 5507229.0
# np sum / size: 5507229.067946667

# with scale-as-you-go
# with float accumulator: 5450137.5
# with double accumulator: 5507228.5

# with scale-after
# with float accumulator: 5546422.5
# with double accumulator: 5507227.5



In [ ]:

print(f"noise magnitude of cropped spectrum: {np.mean(noise_est_slice)/1e6}")
print(f"noise power of cropped spectrum: {np.std(noise_est_slice)/1e6}")


In [ ]:
noise_est_options = pybliss.estimators.noise_power_estimate_options()
noise_est_options.masked_estimate = False
noise_est_options.estimator_method = pybliss.estimators.noise_power_estimator.stddev
noise_stats = pybliss.estimators.estimate_noise_power(flagged_filterbank, noise_est_options)

print(f"noise magnitude of spectrum (estimated without rfi mask): \t{noise_stats.noise_floor/1e6}")
print(f"noise power of spectrum (estimated without rfi mask): \t\t{noise_stats.noise_power/1e6}")
print("---")

noise_est_options = pybliss.estimators.noise_power_estimate_options()
noise_est_options.masked_estimate = True
noise_est_options.estimator_method = pybliss.estimators.noise_power_estimator.stddev
noise_stats = pybliss.estimators.estimate_noise_power(flagged_filterbank, noise_est_options)

print(f"noise magnitude of spectrum (estimated using rfi mask): \t{noise_stats.noise_floor/1e6}")
print(f"noise power of spectrum (estimated using rfi mask): \t\t{noise_stats.noise_power/1e6}")
print("---")

noise_est_options = pybliss.estimators.noise_power_estimate_options()
noise_est_options.masked_estimate = False
noise_est_options.estimator_method = pybliss.estimators.noise_power_estimator.mad
noise_stats = pybliss.estimators.estimate_noise_power(flagged_filterbank.data, noise_est_options)

print(f"noise magnitude of spectrum (estimated using MAD w/o mask): \t{noise_stats.noise_floor/1e6}")
print(f"noise power of spectrum (estimated using MAD w/o mask): \t{noise_stats.noise_power/1e6}")
print("---")


In [ ]:
noise_est_options = pybliss.estimators.noise_power_estimate_options()
noise_est_options.masked_estimate = False
noise_est_options.estimator_method = pybliss.estimators.noise_power_estimator.stddev
noise_stats = pybliss.estimators.estimate_noise_power(np.array(noise_est_slice), noise_est_options)

print(noise_stats.noise_floor)


In [ ]:
opts = pybliss.integrate_drifts_options()
opts.desmear = True
opts.low_rate = 0
opts.high_rate = 48
opts.rate_step_size = 1

dedrift = pybliss.drift_search.integrate_drifts(flagged_filterbank, opts)

In [ ]:
det_plane_with_smear_correction = np.from_dlpack(dedrift.dedrifted_spectrum())

print(det_plane_with_smear_correction.shape)

In [ ]:

voyager_expect = 660000

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)


ax1.set_title("1d detection down max(time) with smear correction")
ax1.plot(det_plane_with_smear_correction[:, voyager_expect-10000:voyager_expect+10000].max(0)/16)
plt.margins(0)

ax2.set_title("2d detection plane with smear correction")
ax2.imshow(det_plane_with_smear_correction[:, voyager_expect-10000:voyager_expect+10000], aspect="auto", interpolation=None, cmap="turbo")




In [ ]:
plt.figure()
_ = plt.hist(noise_est_slice.flatten(), bins=int(np.power(300000, 1/3)))